In [40]:
%load_ext autoreload
%autoreload 2

import sys
import os
import math
import numpy as np
import datetime
from zipfile import ZipFile

sys.path.insert(0, '..')

import casadi as ca
from pyecca.lie.se2 import SE2
from pathlib import Path


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
def derive_se2():
    # derive se2_error
    p = ca.SX.sym('p', 3)
    r = ca.SX.sym('r', 3)
    se2_error = ca.Function('se2_error', [p, r], [SE2.log(SE2.product(SE2.inv(p), r))])

    # SE(2) differential correction term
    e = ca.SX.sym('e', 3)
    x = e[0]
    y = e[1]
    theta = e[2]   

    sin_theta = ca.sin(theta)
    cos_theta = ca.cos(theta)

    sin_2theta = 2*sin_theta*cos_theta
    cos_2theta = 2*cos_theta*cos_theta - 1

    a = ca.if_else(ca.fabs(theta) > 1e-3, theta*sin_theta/(2*(cos_theta - 1)), -1 + theta**2/12 + theta**4/720)
    b = theta/2
    c = ca.if_else(ca.fabs(theta) > 1e-3,(theta*x*sin_theta + (1 - cos_theta)*(theta*y - 2*x))/(2*theta*(1 - cos_theta)), y/2 - theta*x/12 + theta**3*x/720)
    d = ca.if_else(ca.fabs(theta) > 1e-3, (theta*y*sin_theta + (1 - cos_theta)*(-theta*x - 2*y))/(2*theta*(1 - cos_theta)), -x/2 - theta*y/12 - theta**3*y/720)
    
    U = ca.SX.zeros(3, 3)
    U[0, 0] = a
    U[0, 1] = -b
    U[0, 2] = c
    U[1, 0] = b
    U[1, 1] = a
    U[1, 2] = d
    U[2, 0] = 0
    U[2, 1] = 0
    U[2, 2] = -1

    # SE(2) differential correction inverse
    a = ca.if_else(ca.fabs(theta) > 1e-3, sin_theta/theta, 1 - theta**2/6 + theta**4/120)
    b = ca.if_else(ca.fabs(theta) > 1e-3, -(1  - cos_theta)/theta, theta/2 - theta**3/24)
    c = ca.if_else(ca.fabs(theta) > 1e-3, -(x*(theta*cos_theta - theta + sin_theta - sin_2theta/2) + y*(2*cos_theta - cos_2theta/2 - 3/2))/(theta**2*(1 - cos_theta)),
                y/2 + theta*x/6 - theta**2*y/24 - theta**3*x/120 + theta**4*y/720)
    d = ca.if_else(ca.fabs(theta) > 1e-3, -(x*(-2*cos_theta + cos_2theta/2 + 3/2) + y*(theta*cos_theta - theta + sin_theta - sin_2theta/2))/(theta**2*(1 - cos_theta)),
                -x/2 + theta*y/6 + theta**2*x/24 - theta**3*y/120 - theta**4*x/720)
    U_inv = ca.SX.zeros(3, 3)
    U_inv[0, 0] = a
    U_inv[0, 1] = b
    U_inv[0, 2] = c
    U_inv[1, 0] = -b
    U_inv[1, 1] = a
    U_inv[1, 2] = d
    U_inv[2, 2] = 1
    U_inv = -U_inv
    
    # SE(2) U inv series form
    ad_zeta = ca.SX.zeros(3, 3)
    ad_zeta[0, 1] = -theta
    ad_zeta[1, 0] = theta
    ad_zeta[0, 2] = y
    ad_zeta[1, 2] = -x
    n_series = 100

    U_inv_series = ca.SX.eye(3)
    ad_zeta_k = ca.SX.eye(3)
    for k in range(1, 10):
        ad_zeta_k = ad_zeta_k@ad_zeta
        U_inv_series += ad_zeta_k/math.factorial(k+1)
    U_inv_series = -U_inv_series
    
    f_U = ca.Function('se2_U', [e], [U])
    f_U_inv = ca.Function('se2_U_inv', [e], [U_inv])
    f_U_inv_series = ca.Function('se2_U_inv_series', [e], [U_inv_series])

    assert np.max(f_U([0.1, 0.2, 3])@f_U_inv([0.1, 0.2, 3]) - np.eye(3)) < 1e-10

    return {
        'se2_U': f_U,
        'se2_U_inv': f_U_inv,
        'se2_error': se2_error
    }

In [42]:
def generate_code():
    code_opts = {
        'with_header': True,
        'with_mem': False,
        'main': False,
        'with_import': False,
        'with_export': False,
        'with_import': False
    }

    start_dir = Path.cwd()
    gen_dir = start_dir / "gen"
    gen_dir.mkdir(parents=True, exist_ok=True)

    os.chdir(gen_dir)
    gen_bezier6 = ca.CodeGenerator('se2', code_opts)
    diff_correction = derive_se2()
    gen_bezier6.add(diff_correction['se2_U'])
    gen_bezier6.add(diff_correction['se2_U_inv'])
    gen_bezier6.add(diff_correction['se2_error'])
    gen_bezier6.generate()

    version=datetime.datetime.now().strftime("%m-%d-%Y-%H:%M")
    with ZipFile(gen_dir / f'gen_se2-{version}.zip', 'w') as myzip:
        for f in Path('.').glob("*.c"):
            myzip.write(f)
            f.unlink()
        for f in Path('.').glob("*.h"):
            myzip.write(f)
            f.unlink()
    os.chdir(start_dir)
    
generate_code()